<a href="https://colab.research.google.com/github/jorgemunozl/vla-test/blob/main/TEST/thirteen_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thirteen Test - Test our processor, CUDA nedded?

Let's see if our processor is good.

## Setup

The following cells set up the environment:
1. Clone the XHUMAN repository
2. Install dependencies
3. Authenticate with HuggingFace Hub
4. Import required libraries

In [1]:
# Clone

In [2]:
%cd XHUMAN

/content/XHUMAN


In [3]:
!uv pip install -e .[pi]

Using Python 3.12.12 environment at: /usr
Resolved 215 packages in 30.67s
Prepared 45 packages in 21.44s
Uninstalled 14 packages in 969ms
Installed 45 packages in 250ms
 + async-lru==2.1.0
 + av==15.1.0
 + comm==0.2.3
 + deepdiff==8.6.1
 - diffusers==0.36.0
 + diffusers==0.35.2
 + draccus==0.10.0
 + evdev==1.9.2
 + faker==40.1.2
 + feetech-servo-sdk==1.0.0
 - huggingface-hub==0.36.0
 + huggingface-hub==0.35.3
 + inquirerpy==0.3.4
 - ipykernel==6.17.1
 + ipykernel==7.1.0
 + jedi==0.19.2
 + json5==0.13.0
 + jsonlines==4.0.0
 + jupyter==1.1.1
 - jupyter-client==7.4.9
 + jupyter-client==8.8.0
 + jupyter-lsp==2.3.0
 + jupyterlab==4.5.2
 + jupyterlab-server==2.28.0
 + lerobot==0.4.4 (from git+https://github.com/huggingface/lerobot.git@6d34a986de44c5f22a9a99ed514f1b16832c3f32)
 + mergedeep==1.3.4
 + mypy-extensions==1.1.0
 - notebook==6.5.7
 + notebook==7.5.2
 - nvidia-cudnn-cu12==9.10.2.21
 + nvidia-cudnn-cu12==9.5.1.17
 - nvidia-cusparselt-cu12==0.7.1
 + nvidia-cusparselt-cu12==0.6.3
 - nvi

In [5]:
import time
from contextlib import nullcontext
from typing import Any

import torch
from accelerate import Accelerator
from accelerate.utils import DistributedDataParallelKwargs
from torch.optim import Optimizer

from lerobot.configs import parser
from lerobot.datasets.sampler import EpisodeAwareSampler
from lerobot.datasets.utils import cycle
from lerobot.optim.factory import make_optimizer_and_scheduler
from lerobot.policies.pretrained import PreTrainedPolicy
from lerobot.utils.logging_utils import AverageMeter, MetricsTracker
from lerobot.utils.random_utils import set_seed
from lerobot.utils.train_utils import load_training_state
from lerobot.utils.utils import (
    format_big_number,
    has_method,
    init_logging,
)

from xhuman.policies.factory import make_xhuman_policy, make_xhuman_pre_post_processors
from xhuman.configs.train import TrainPipelineConfigXHUMAN
from xhuman.datasets.factory import make_dataset_xhuman
from xhuman.datasets.utils import split_train_eval_episodes
from xhuman.logger import logger

## Helper Functions

These functions handle dataset loading and policy updates. They are designed to work with distributed training using HuggingFace Accelerate.

In [6]:
def load_dataset(cfg: TrainPipelineConfigXHUMAN, episodes: list[int], is_main_process: bool = True, accelerator: Accelerator | None = None):
    """
    Load the dataset for training and evaluation.
    """
    # Dataset loading synchronization: main process downloads first to avoid race conditions
    cfg.dataset.episodes = episodes

    if is_main_process:
        logger.info("Creating dataset")
        dataset = make_dataset_xhuman(cfg)

    accelerator.wait_for_everyone()

    # Now all other processes can safely load the dataset
    if not is_main_process:
        dataset = make_dataset_xhuman(cfg)

    return dataset

In [7]:
from xhuman.policies.pi05ki.configuration_pi05ki import PI05KIConfig

policy_config = PI05KIConfig(repo_id="none",device="cuda")

## Configuration and Setup

Configure your dataset and policy settings here. The dataset configuration specifies which HuggingFace repository to load, and the policy configuration sets up the PI05 model architecture.

In [8]:
from xhuman.configs.default import LerobotDatasetConfig

dataset_config = LerobotDatasetConfig(
    repo_id="NONHUMAN-RESEARCH/test-general-idx",
)

In [9]:

cfg = TrainPipelineConfigXHUMAN(
    dataset=dataset_config,
    policy=policy_config # Example policy configuration, replace with your actual policy path
)
cfg.validate()

## Training Setup

Initialize the Accelerator for distributed training and set up the training environment. The accelerator automatically handles:
- Multi-GPU training
- Mixed precision training
- Gradient synchronization across processes

In [10]:
# Create Accelerator
# It will automatically detect if running in distributed mode or single-process mode
# We set step_scheduler_with_optimizer=False to prevent accelerate from adjusting the lr_scheduler steps based on the num_processes
# We set find_unused_parameters=True to handle models with conditional computation
ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)
accelerator = Accelerator(step_scheduler_with_optimizer=False, kwargs_handlers=[ddp_kwargs])

init_logging(accelerator=accelerator)

# Determine if this is the main process (for logging and checkpointing)
is_main_process = accelerator.is_main_process

# Set seed if specified
if cfg.seed is not None:
    set_seed(cfg.seed, accelerator=accelerator)

# Use accelerator's device
device = accelerator.device
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

In [32]:
import lerobot.datasets.lerobot_dataset
from lerobot.datasets.utils import load_nested_dataset as original_load_nested_dataset
import datasets

# Monkey patch to handle schema mismatch
# The dataset parquet files have a schema that conflicts with the strict fixed-length definition
# expected by the code. We patch the loader to ignore strict features if loading fails.
def patched_load_nested_dataset(pq_dir, features=None, episodes=None):
    try:
        return original_load_nested_dataset(pq_dir, features=features, episodes=episodes)
    except (datasets.builder.DatasetGenerationError, datasets.table.CastError, Exception) as e:
        # Fallback if strict schema casting fails
        print(f"Warning: Dataset loading failed with strict features ({type(e).__name__}). Retrying without features constraint.")
        return original_load_nested_dataset(pq_dir, features=None, episodes=episodes)

lerobot.datasets.lerobot_dataset.load_nested_dataset = patched_load_nested_dataset

DEBUG_MODE = True  # Set to False for full training
DEBUG_MAX_EPISODES = 3  # Use only first N episodes for debugging

# First, get total episodes count (load minimal dataset to check)
if is_main_process:
    temp_dataset = make_dataset_xhuman(cfg)
    total_episodes = temp_dataset.meta.total_episodes
    del temp_dataset
    logger.info(f"Total episodes available: {total_episodes}")
else:
    # For non-main processes, use a reasonable default
    # In practice, this will be synced after main process loads
    total_episodes = 4  # Fallback - adjust if needed

accelerator.wait_for_everyone()

# Limit episodes for debugging
if DEBUG_MODE:
    episodes = list(range(min(DEBUG_MAX_EPISODES, total_episodes)))
    if is_main_process:
        logger.info(f"DEBUG MODE: Using only {len(episodes)} episodes")
else:
    episodes = list(range(total_episodes))

# Split episodes
train_episodes, eval_episodes = split_train_eval_episodes(
    episodes, split_ratio=cfg.split_ratio, seed=42
)

# Load dataset with ONLY train episodes (proper way to filter)
# This uses the load_dataset helper function which sets cfg.dataset.episodes
if is_main_process:
    logger.info(f"Loading train dataset with {len(train_episodes)} episodes")
dataset = load_dataset(cfg, train_episodes, is_main_process=is_main_process, accelerator=accelerator)
dataset.train_with_subtasks = True

[19:51:17] INFO     Total episodes available: 437                                     ]8;id=230283;file:///tmp/ipython-input-426660049.py\ipython-input-426660049.py]8;;\:]8;id=717870;file:///tmp/ipython-input-426660049.py#26\26]8;;\

           INFO     DEBUG MODE: Using only 3 episodes                                 ]8;id=240174;file:///tmp/ipython-input-426660049.py\ipython-input-426660049.py]8;;\:]8;id=861722;file:///tmp/ipython-input-426660049.py#38\38]8;;\

           INFO     Loading train dataset with 2 episodes                             ]8;id=146316;file:///tmp/ipython-input-426660049.py\ipython-input-426660049.py]8;;\:]8;id=772246;file:///tmp/ipython-input-426660049.py#50\50]8;;\

           INFO     Creating dataset                                                  ]8;id=571858;file:///tmp/ipython-input-1989669429.py\ipython-input-1989669429.py]8;;\:]8;id=91161;file:///tmp/ipython-input-1989669429.py#9\9]8;;\

## Dataset and Model Information

Display metadata about the loaded dataset and model. This includes:
- Total number of episodes and frames
- Model parameter counts
- Effective batch size (accounting for distributed training)

In [33]:
# Display dataset metadata and model configuration
if is_main_process:
    from pprint import pprint

    print("=" * 80)
    print("DATASET METADATA")
    print("=" * 80)
    print(f"\nDataset Repository: {dataset.repo_id}")
    print(f"Total Episodes: {dataset.meta.total_episodes}")
    print(f"Training Episodes: {len(train_episodes)}")
    print(f"Number of Frames: {dataset.num_frames:,}")
    print(f"Number of Episodes (loaded): {dataset.num_episodes}")


    print("\n" + "=" * 80)

DATASET METADATA

Dataset Repository: NONHUMAN-RESEARCH/test-general-idx
Total Episodes: 437
Training Episodes: 2
Number of Frames: 2,251
Number of Episodes (loaded): 2



In [64]:
# Create processors
preprocessor, postprocessor = make_xhuman_pre_post_processors(
    policy_cfg=cfg.policy,
    pretrained_path=cfg.policy.pretrained_path,
)

In [65]:
# Print training info
if is_main_process:
    logger.info(f"Output dir: {cfg.output_dir}")
    logger.info(f"Steps: {cfg.steps} ({format_big_number(cfg.steps)})")
    logger.info(f"Dataset frames: {dataset.num_frames} ({format_big_number(dataset.num_frames)})")
    logger.info(f"Dataset episodes: {dataset.num_episodes}")
    num_processes = accelerator.num_processes
    effective_bs = cfg.batch_size * num_processes
    logger.info(f"Effective batch size: {cfg.batch_size} x {num_processes} = {effective_bs}")

[20:15:21] INFO     Output dir: outputs/train/2026-01-22/19-39-50_pi05_ki             ]8;id=584004;file:///tmp/ipython-input-1805760154.py\ipython-input-1805760154.py]8;;\:]8;id=230283;file:///tmp/ipython-input-1805760154.py#3\3]8;;\

           INFO     Steps: 100000 (100K)                                              ]8;id=813694;file:///tmp/ipython-input-1805760154.py\ipython-input-1805760154.py]8;;\:]8;id=58655;file:///tmp/ipython-input-1805760154.py#4\4]8;;\

           INFO     Dataset frames: 2251 (2K)                                         ]8;id=330776;file:///tmp/ipython-input-1805760154.py\ipython-input-1805760154.py]8;;\:]8;id=420651;file:///tmp/ipython-input-1805760154.py#5\5]8;;\

           INFO     Dataset episodes: 2                                               ]8;id=988712;file:///tmp/ipython-input-1805760154.py\ipython-input-1805760154.py]8;;\:]8;id=594731;file:///tmp/ipython-input-1805760154.py#6\6]8;;\

           INFO     Effective batch size: 8 x 1 = 8                                   ]8;id=687277;file:///tmp/ipython-input-1805760154.py\ipython-input-1805760154.py]8;;\:]8;id=523481;file:///tmp/ipython-input-1805760154.py#9\9]8;;\

In [66]:
# Create dataloader
if hasattr(cfg.policy, "drop_n_last_frames"):
    logger.info(f"Dropping {cfg.policy.drop_n_last_frames} last frames")
    shuffle = False
    sampler = EpisodeAwareSampler(
        dataset.meta.episodes["dataset_from_index"],
        dataset.meta.episodes["dataset_to_index"],
        drop_n_last_frames=cfg.policy.drop_n_last_frames,
        shuffle=True,
    )
else:
    logger.info("Not dropping any frames")
    shuffle = True
    sampler = None

[20:15:22] INFO     Not dropping any frames                                           ]8;id=481141;file:///tmp/ipython-input-988475408.py\ipython-input-988475408.py]8;;\:]8;id=149811;file:///tmp/ipython-input-988475408.py#12\12]8;;\

In [67]:
# Training initialization
# This logs the start of training and shows how many episodes will be used
if is_main_process:
    logger.info("Start offline training on a fixed dataset")
    logger.info(f"Train episodes: {len(train_episodes)}")
    logger.info(f"Total training steps: {cfg.steps}")

[20:15:25] INFO     Start offline training on a fixed dataset                         ]8;id=588637;file:///tmp/ipython-input-2557974599.py\ipython-input-2557974599.py]8;;\:]8;id=565158;file:///tmp/ipython-input-2557974599.py#4\4]8;;\

           INFO     Train episodes: 2                                                 ]8;id=941435;file:///tmp/ipython-input-2557974599.py\ipython-input-2557974599.py]8;;\:]8;id=611878;file:///tmp/ipython-input-2557974599.py#5\5]8;;\

           INFO     Total training steps: 100000                                      ]8;id=534277;file:///tmp/ipython-input-2557974599.py\ipython-input-2557974599.py]8;;\:]8;id=517488;file:///tmp/ipython-input-2557974599.py#6\6]8;;\

In [68]:
from torch.utils.data import Dataset

class SmartSubset(Dataset):
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices

    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]]

    def __len__(self):
        return len(self.indices)

    def __getattr__(self, name):
        # This is the magic part:
        # If the code asks for 'meta', 'fps', etc., and this class doesn't have it,
        # it automatically looks inside the original dataset.
        return getattr(self.dataset, name)

# --- USAGE ---
# Use SmartSubset instead of torch.utils.data.Subset
debug_subset = SmartSubset(dataset, range(0, 50))

# Now create your loader normally
train_dataloader = torch.utils.data.DataLoader(
    debug_subset,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

In [86]:
train_dataloader

In [69]:
dl_iter = iter(train_dataloader)

In [62]:
import sys
import importlib

# Polyfill 'imp' for Python 3.12 compatibility
if "imp" not in sys.modules:
    import types
    imp = types.ModuleType("imp")
    imp.reload = importlib.reload
    sys.modules["imp"] = imp

%load_ext autoreload
%autoreload 2

# Call Subtask Processor
Important track the index for loss mask

In [76]:
frames = next(dl_iter)

[20:28:10] INFO     Loaded item with index 2                                                  ]8;id=988210;file:///content/XHUMAN/xhuman/datasets/xhuman_dataset.py\xhuman_dataset.py]8;;\:]8;id=12038;file:///content/XHUMAN/xhuman/datasets/xhuman_dataset.py#190\190]8;;\

           INFO     Loaded item with index 40                                                 ]8;id=787352;file:///content/XHUMAN/xhuman/datasets/xhuman_dataset.py\xhuman_dataset.py]8;;\:]8;id=279786;file:///content/XHUMAN/xhuman/datasets/xhuman_dataset.py#190\190]8;;\

In [77]:
frames.keys()

dict_keys(['observation.images.left', 'observation.images.top', 'observation.images.right', 'action', 'observation.state', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index', 'general_task_index', 'action_is_pad', 'task', 'general_task', 'train_with_subtask'])

In [79]:
frames["subtask"] = frames["task"]

In [80]:
batch = preprocessor.subtask(frames)

In [81]:
batch.keys()

dict_keys(['action', 'next.reward', 'next.done', 'next.truncated', 'info', 'action_is_pad', 'task', 'index', 'task_index', 'episode_index', 'subtask', 'subtask_tokens', 'observation.images.left', 'observation.images.top', 'observation.images.right', 'observation.state', 'observation.language.tokens', 'observation.language.attention_mask'])

In [82]:
sub, tokens  = batch["subtask_tokens"], batch["observation.language.tokens"]

In [84]:
sublang = tokenize.batch_decode(sub,skip_special_tokens=True)
sublang

['pick up the strawberry and put it in the basket',
 'pick up the strawberry and put it in the basket']

In [85]:
tokenslang = tokenize.batch_decode(tokens,skip_special_tokens=True)
tokenslang


['Task: pick up the strawberry and put it in the basket. Subtask: ',
 'Task: pick up the strawberry and put it in the basket. Subtask: ']

# Test loss calculation

In [58]:
tokens_id = "observation.language.tokens"
mask_id = "observation.language.attention_mask"

tokens, masks = batch[tokens_id], batch[mask_id]
noise = None
time = None
# Processor should have a method to get the subtask tokenized
# List of tensors


In [51]:
from transformers import AutoTokenizer

tokenize = AutoTokenizer.from_pretrained("google/paligemma-3b-pt-224")
stokens = tokenize.batch_decode(tokens)

In [60]:
stokens

['<bos>Task: pick up the strawberry and put it in the basket. Subtask: <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<bos>Task: pic